stellargraph is only compatible with python 3.8 so we downgrade python
then we need to re-install pip and import some other libraries to be able to install stellargraph with pip and import it

In [1]:
!pip install chardet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 6.9 MB/s eta 0:00:00


In [2]:
pip install stellargraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 435.2/435.2 kB 11.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tfx-bsl 1.12.0 requires google-api-python-client<2,>=1.7.11, but you have google-api-python-client 2.83.0 which is incompatible.
onnx 1.13.1 requires protobuf<4,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
apache-beam 2.44.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import os
import keras
import stellargraph as sg
from stellargraph.mapper import FullBatchNodeGenerator
from stellargraph.layer import GCN
from stellargraph.layer import GAT
from stellargraph.mapper import GraphSAGENodeGenerator
from stellargraph.layer import GraphSAGE
from stellargraph.datasets import Cora
from stellargraph import StellarGraph
from tensorflow.keras import layers, optimizers, losses, metrics, Model,regularizers
from sklearn import preprocessing, model_selection
from IPython.display import display, HTML
import matplotlib.pyplot as plt
%matplotlib inline

from keras.initializers import GlorotUniform
import numpy as np
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

# Data Prep

building the cora. stellargraph from the dataset instead of using the built-in cora tocostumize the node feautures 

In [4]:
cora = Cora()
cora.download()

# the base_directory property tells us where it was downloaded to:
cora_cites_file = os.path.join(cora.base_directory, "cora.cites")
#we need make changes to cora.content
#a row of cora.content: ID+1433 o or 1+ label
cora_content_file = os.path.join(cora.base_directory, "cora.content")

 **loading nodes**

We’ve now got the files on disk, so we can read them using the pd.read_csv function. Despite the “CSV” in the name, this function can be used to read TSV files too. The files don’t have a row of column headings, so we’ll want to set our own.

First, the edges. We can use source and target as the column headings, to match StellarGraph’s defaults. However, the natural phrasing is “paper X cites paper Y”, not “paper Y is cited by paper X”, so we use the columns in reverse order to match.

In [5]:
cora_cites = pd.read_csv(
    cora_cites_file,
    sep="\t",  # tab-separated
    header=None,  # no heading row
    names=["target", "source"],  # set our own names for the columns
)
cora_cites

target   source
0         35     1033
1         35   103482
2         35   103515
3         35  1050679
4         35  1103960
...      ...      ...
5424  853116    19621
5425  853116   853155
5426  853118  1140289
5427  853155   853118
5428  954315  1155073

[5429 rows x 2 columns]

In [6]:
print(cora_cites.iloc[0])

target      35
source    1033
Name: 0, dtype: int64


**loading features**

creating the features only using graph information

Now, the nodes. Again, we have to choose the columns’ names. The names of the 0/1-columns don’t matter so much, but we can give the first column (of IDs) and the last one (of subjects) useful names.

In [7]:
cora_feature_names = [f"w{i}" for i in range(1433)]

cora_raw_content = pd.read_csv(
    cora_content_file,
    sep="\t",  # tab-separated
    header=None,  # no heading row
    names=["id", *cora_feature_names, "subject"],  # set our own names for the columns
)
cora_raw_content

id  w0  w1  w2  w3  w4  w5  w6  w7  w8  ...  w1424  w1425  w1426  \
0       31336   0   0   0   0   0   0   0   0   0  ...      0      0      1   
1     1061127   0   0   0   0   0   0   0   0   0  ...      0      1      0   
2     1106406   0   0   0   0   0   0   0   0   0  ...      0      0      0   
3       13195   0   0   0   0   0   0   0   0   0  ...      0      0      0   
4       37879   0   0   0   0   0   0   0   0   0  ...      0      0      0   
...       ...  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...    ...    ...    ...   
2703  1128975   0   0   0   0   0   0   0   0   0  ...      0      0      0   
2704  1128977   0   0   0   0   0   0   0   0   0  ...      0      0      0   
2705  1128978   0   0   0   0   0   0   0   0   0  ...      0      0      0   
2706   117328   0   0   0   0   1   0   0   0   0  ...      0      0      0   
2707    24043   0   0   0   0   0   0   0   0   0  ...      0      0      0   

      w1427  w1428  w1429  w1430  w1431  w1432                 subject  
0         0      0      0      0      0      0         Neural_Networks  
1         0      0      0      0      0      0           Rule_Learning  
2         0      0      0      0      0      0  Reinforcement_Learning  
3         0      0      0      0      0      0  Reinforcement_Learning  
4         0      0      0      0      0      0   Probabilistic_Methods  
...     ...    ...    ...    ...    ...    ...                     ...  
2703      0      0      0      0      0      0      Genetic_Algorithms  
2704      0      0      0      0      0      0      Genetic_Algorithms  
2705      0      0      0      0      0      0      Genetic_Algorithms  
2706      0      0      0      0      0      0              Case_Based  
2707      0      0      0      0      0      0         Neural_Networks  

[2708 rows x 1435 columns]

As we saw above when adding node features, StellarGraph uses the index of the DataFrame as the connection between a node and a row of the DataFrame. Currently our dataframe just has a simple numeric range as the index, but it needs to be using the id column. Pandas offers a few ways to control the indexing; in this case, we want to replace the current index by moving the id column to it, which is done most easily with set_index:

In [8]:
cora_content_str_subject = cora_raw_content.set_index("id")
cora_content_str_subject

w0  w1  w2  w3  w4  w5  w6  w7  w8  w9  ...  w1424  w1425  w1426  \
id                                               ...                        
31336     0   0   0   0   0   0   0   0   0   0  ...      0      0      1   
1061127   0   0   0   0   0   0   0   0   0   0  ...      0      1      0   
1106406   0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
13195     0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
37879     0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
...      ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...    ...    ...    ...   
1128975   0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
1128977   0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
1128978   0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
117328    0   0   0   0   1   0   0   0   0   0  ...      0      0      0   
24043     0   0   0   0   0   0   0   0   0   0  ...      0      0      0   

         w1427  w1428  w1429  w1430  w1431  w1432                 subject  
id                                                                         
31336        0      0      0      0      0      0         Neural_Networks  
1061127      0      0      0      0      0      0           Rule_Learning  
1106406      0      0      0      0      0      0  Reinforcement_Learning  
13195        0      0      0      0      0      0  Reinforcement_Learning  
37879        0      0      0      0      0      0   Probabilistic_Methods  
...        ...    ...    ...    ...    ...    ...                     ...  
1128975      0      0      0      0      0      0      Genetic_Algorithms  
1128977      0      0      0      0      0      0      Genetic_Algorithms  
1128978      0      0      0      0      0      0      Genetic_Algorithms  
117328       0      0      0      0      0      0              Case_Based  
24043        0      0      0      0      0      0         Neural_Networks  

[2708 rows x 1434 columns]

In [9]:
print(len(cora_raw_content))

2708


remove subject from content

In [10]:
cora_content_no_subject = cora_content_str_subject.drop(columns="subject")
cora_content_no_subject

w0  w1  w2  w3  w4  w5  w6  w7  w8  w9  ...  w1423  w1424  w1425  \
id                                               ...                        
31336     0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
1061127   0   0   0   0   0   0   0   0   0   0  ...      0      0      1   
1106406   0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
13195     0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
37879     0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
...      ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...    ...    ...    ...   
1128975   0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
1128977   0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
1128978   0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
117328    0   0   0   0   1   0   0   0   0   0  ...      1      0      0   
24043     0   0   0   0   0   0   0   0   0   0  ...      0      0      0   

         w1426  w1427  w1428  w1429  w1430  w1431  w1432  
id                                                        
31336        1      0      0      0      0      0      0  
1061127      0      0      0      0      0      0      0  
1106406      0      0      0      0      0      0      0  
13195        0      0      0      0      0      0      0  
37879        0      0      0      0      0      0      0  
...        ...    ...    ...    ...    ...    ...    ...  
1128975      0      0      0      0      0      0      0  
1128977      0      0      0      0      0      0      0  
1128978      0      0      0      0      0      0      0  
117328       0      0      0      0      0      0      0  
24043        0      0      0      0      0      0      0  

[2708 rows x 1433 columns]

the newe feature vector

coloring: a node and 10 of its neghbors use the same color so we have 270 groups in 270 colors

plot the colored graph

In [11]:
import networkx as nx
import matplotlib.pyplot as plt
num_of_groups=10
# Create a stellargraph
graph = StellarGraph({"paper": cora_content_no_subject}, {"cites": cora_cites})
G = graph.to_networkx()

# Calculate the number of neighbors for each node
num_neighbors = dict(G.degree)

# Divide the nodes into groups of 10 neighbors each
groups = {}

group_count = 0
for node, num in num_neighbors.items():
    group = group_count // num_of_groups
    if group not in groups:
        groups[group] = []
    groups[group].append(node)
    group_count += 1
#print(groups)
# Assign a unique color to each group of nodes
colors = {}
for i, group in enumerate(groups.values()):
    for node in group:
        colors[node] = i

# Visualize the stellargraph with the assigned colors for the nodes
#nx.draw(G, node_color=[colors[node] for node in G.nodes], cmap=plt.get_cmap("Set1"), with_labels=True)
#plt.show()

color is a dictinary where. each ley is a node_id and value is a number indicating the color (0 to 270) in this cell we convert (0,270) to binary numbers

In [12]:
import numpy as np
# Loop through the keys of the dictionary
temp=int(len(cora_raw_content)/num_of_groups)+1
max_digits=len(np.binary_repr(temp))
for key in colors.keys():
    # Convert the value to binary string and remove the '0b' prefix
    #binary_value = bin(colors[key])[2:]
    binary_value=np.array([np.binary_repr(colors[key],width=max_digits)])
    # Update the dictionary with the new binary value
    colors[key] = binary_value

In [13]:
#print(colors)

turning dictionary to dataframe

In [14]:
import numpy as np



df = pd.DataFrame.from_dict(colors, orient='index', columns=['color'])
df.index.name='id'


# Split each binary string into separate columns
df = df['color'].apply(lambda x: pd.Series(list(x)))

# Rename the columns to represent the position of each digit
df.columns = [f"c_{i}" for i in range(len(df.columns))]
#df.insert(0,'id', cora_raw_content['id'] )
# Print the DataFrame
#cora_raw_content['id']
#df2=df.set_index("id")
#print(df2)


In [15]:
are_columns_same = (cora_content_no_subject.index == df.index).all()

# Print the result
if are_columns_same:
    print("The columns are the same")
else:
    print("The columns are different")

The columns are the same


In [16]:
df

c_0 c_1 c_2 c_3 c_4 c_5 c_6 c_7 c_8
id                                         
31336     0   0   0   0   0   0   0   0   0
1061127   0   0   0   0   0   0   0   0   0
1106406   0   0   0   0   0   0   0   0   0
13195     0   0   0   0   0   0   0   0   0
37879     0   0   0   0   0   0   0   0   0
...      ..  ..  ..  ..  ..  ..  ..  ..  ..
1128975   1   0   0   0   0   1   1   1   0
1128977   1   0   0   0   0   1   1   1   0
1128978   1   0   0   0   0   1   1   1   0
117328    1   0   0   0   0   1   1   1   0
24043     1   0   0   0   0   1   1   1   0

[2708 rows x 9 columns]

In [17]:
cora_content_no_subject

w0  w1  w2  w3  w4  w5  w6  w7  w8  w9  ...  w1423  w1424  w1425  \
id                                               ...                        
31336     0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
1061127   0   0   0   0   0   0   0   0   0   0  ...      0      0      1   
1106406   0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
13195     0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
37879     0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
...      ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ...    ...    ...    ...   
1128975   0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
1128977   0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
1128978   0   0   0   0   0   0   0   0   0   0  ...      0      0      0   
117328    0   0   0   0   1   0   0   0   0   0  ...      1      0      0   
24043     0   0   0   0   0   0   0   0   0   0  ...      0      0      0   

         w1426  w1427  w1428  w1429  w1430  w1431  w1432  
id                                                        
31336        1      0      0      0      0      0      0  
1061127      0      0      0      0      0      0      0  
1106406      0      0      0      0      0      0      0  
13195        0      0      0      0      0      0      0  
37879        0      0      0      0      0      0      0  
...        ...    ...    ...    ...    ...    ...    ...  
1128975      0      0      0      0      0      0      0  
1128977      0      0      0      0      0      0      0  
1128978      0      0      0      0      0      0      0  
117328       0      0      0      0      0      0      0  
24043        0      0      0      0      0      0      0  

[2708 rows x 1433 columns]

In [18]:
cora_subject = cora_content_str_subject["subject"]
cora_subject

id
31336             Neural_Networks
1061127             Rule_Learning
1106406    Reinforcement_Learning
13195      Reinforcement_Learning
37879       Probabilistic_Methods
                    ...          
1128975        Genetic_Algorithms
1128977        Genetic_Algorithms
1128978        Genetic_Algorithms
117328                 Case_Based
24043             Neural_Networks
Name: subject, Length: 2708, dtype: object

We’ve got all the right node data, and the right edges, so now we can create a StellarGraph using the techniques we saw in the “homogeneous graph with features” section.

In [19]:
cora_no_subject = StellarGraph({"paper": df}, {"cites": cora_cites})
print(cora_no_subject.info())

StellarGraph: Undirected multigraph
 Nodes: 2708, Edges: 5429

 Node types:
  paper: [2708]
    Features: float32 vector, length 9
    Edge types: paper-cites->paper

 Edge types:
    paper-cites->paper: [5429]
        Weights: all 1 (default)
        Features: none


In [20]:
print(len(cora_cites))

5429


renaming--**rest follows like library**

In [21]:
G=cora_no_subject
node_subjects=cora_subject

In [22]:
def my_gcn(G,node_subjects):
    #splitting the data : 140 nodes for training, 500 for validation and 2068 for testing
    train_subjects, test_subjects = model_selection.train_test_split(node_subjects, train_size=140, test_size=None, stratify=node_subjects)
    val_subjects, test_subjects = model_selection.train_test_split(test_subjects, train_size=500, test_size=None, stratify=test_subjects)
    #one hot encoding for node subjects
    target_encoding = preprocessing.LabelBinarizer()
    train_targets = target_encoding.fit_transform(train_subjects)
    val_targets = target_encoding.transform(val_subjects)
    test_targets = target_encoding.transform(test_subjects)
    #creating GCN layers
    #using full batch node generator to convert graph structure and node features into a format that can be fed into a keras model
    generator = FullBatchNodeGenerator(G, method="gcn")
    #matching nodes and actual labels to create model input
    train_gen = generator.flow(train_subjects.index, train_targets)
    #specifying the machine learning model: GCN with 2 hidden layers (16 units each) Relu for activation function dropout rate 50%
    gcn = GCN(layer_sizes=[16, 16], activations=["relu", "relu"], generator=generator, dropout=0.5)
    #input and output tensors
    # x_out is <KerasTensor: shape=(1, None, 16) dtype=float32 (created by layer 'gather_indices_1')>
    x_inp, x_out = gcn.in_out_tensors()
    #Predictions of each node’s class/subject will be computed from x_out (tensor that holds a 16-dim vector for nodes requested in traing/prediction)
    #using an additional dense layer (with one unit per class) with softmax activation.
    #softmax ensures that the final outputs for each input node will be a vector of probabilities( all values between 0 and 1, vector sum is 1).
    #Predicted class is the element with the highest value.
    initializer = keras.initializers.GlorotUniform()
    predictions = layers.Dense(units=train_targets.shape[1], activation="softmax",kernel_initializer=initializer)(x_out)
    #training
    #creating the Keras model (to compute/predict 16 dim node embedding vecors, we can swap predictions to x_out and use dim reduction for visualizing)
    
    model = Model(inputs=x_inp, outputs=predictions)
    model.compile(
    optimizer=optimizers.Adam(learning_rate=0.01),
    loss=losses.categorical_crossentropy,
    metrics=["acc",metrics.Precision(),metrics.Recall()],
    )
    #validation and test generator
    val_gen = generator.flow(val_subjects.index, val_targets)
    test_gen = generator.flow(test_subjects.index, test_targets)
    #stop training when validation accuracy stops improving
    es_callback = EarlyStopping(monitor="val_acc", patience=50, restore_best_weights=True)
    #training the model
    history = model.fit(
    train_gen,
    epochs=200,
    validation_data=val_gen,
    verbose=2,
    shuffle=False,  # this should be False, since shuffling data means shuffling the whole graph
    callbacks=[es_callback],
    )
    #testing
    #metrics for trained model
    #The keras.evaluate() function will give you the loss value for every batch. The keras.predict() function will give you the actual predictions for all samples in a batch, for all batches.
    test_metrics = model.evaluate(test_gen)
    for name, val in zip(model.metrics_names, test_metrics):
        print("\t{}: {:0.4f}".format(name, val))
    if test_metrics[2]+test_metrics[3]==0:
        f1_score='inf'
    else:
        f1_score=(2*test_metrics[2]*test_metrics[3])/(test_metrics[2]+test_metrics[3])
    print("f1_score:",f1_score)
    #making predictions
    #predictiong all classes without knowing the subject
    all_nodes = node_subjects.index
    all_gen = generator.flow(all_nodes)
    all_predictions = model.predict(all_gen)
    #inverse transform from one-hot encoding
    node_predictions = target_encoding.inverse_transform(all_predictions.squeeze())
    df = pd.DataFrame({"Predicted": node_predictions, "True": node_subjects})
    print(df)


def my_sgn(G,node_subjects):
    train_subjects, test_subjects = model_selection.train_test_split(
        node_subjects, train_size=140, test_size=None, stratify=node_subjects
    )
    val_subjects, test_subjects = model_selection.train_test_split(
        test_subjects, train_size=500, test_size=None, stratify=test_subjects
    )
    target_encoding = preprocessing.LabelBinarizer()

    train_targets = target_encoding.fit_transform(train_subjects)
    val_targets = target_encoding.transform(val_subjects)
    test_targets = target_encoding.transform(test_subjects)
    generator = FullBatchNodeGenerator(G, method="sgc", k=2)
    train_gen = generator.flow(train_subjects.index, train_targets)
    sgc = GCN(
        layer_sizes=[train_targets.shape[1]],
        generator=generator,
        bias=True,
        dropout=0.5,
        activations=["softmax"],
        kernel_regularizer=regularizers.l2(5e-4),
    )
    x_inp, predictions = sgc.in_out_tensors()
    model = Model(inputs=x_inp, outputs=predictions)
    model.compile(
        optimizer=optimizers.Adam(lr=0.2),
        loss=losses.categorical_crossentropy,
        metrics=["acc",metrics.Precision(),metrics.Recall()],
    )
    val_gen = generator.flow(val_subjects.index, val_targets)
    if not os.path.isdir("logs"):
        os.makedirs("logs")
    es_callback = EarlyStopping(
        monitor="val_acc", patience=50
    )  # patience is the number of epochs to wait before early stopping in case of no further improvement
    mc_callback = ModelCheckpoint(
        "logs/best_model_sgn.h5", monitor="val_acc", save_best_only=True, save_weights_only=True
    )
    history = model.fit(
        train_gen,
        epochs=50,
        validation_data=val_gen,
        verbose=0,
        shuffle=False,  # this should be False, since shuffling data means shuffling the whole graph
        callbacks=[es_callback, mc_callback],
    )
    model.load_weights("logs/best_model_sgn.h5")
    test_gen = generator.flow(test_subjects.index, test_targets)
    test_metrics = model.evaluate(test_gen)
    for name, val in zip(model.metrics_names, test_metrics):
        print("\t{}: {:0.4f}".format(name, val))
    if test_metrics[2]+test_metrics[3]==0:
        f1_score='inf'
    else:
        f1_score=(2*test_metrics[2]*test_metrics[3])/(test_metrics[2]+test_metrics[3])
    print("f1_score:",f1_score)

def my_gat(G,node_subjects):
    train_subjects, test_subjects = model_selection.train_test_split(
        node_subjects, train_size=140, test_size=None, stratify=node_subjects
    )
    val_subjects, test_subjects = model_selection.train_test_split(
        test_subjects, train_size=500, test_size=None, stratify=test_subjects
    )
    target_encoding = preprocessing.LabelBinarizer()

    train_targets = target_encoding.fit_transform(train_subjects)
    val_targets = target_encoding.transform(val_subjects)
    test_targets = target_encoding.transform(test_subjects)
    generator = FullBatchNodeGenerator(G, method="gat")
    train_gen = generator.flow(train_subjects.index, train_targets)
    gat = GAT(
        layer_sizes=[8, train_targets.shape[1]],
        activations=["elu", "softmax"],
        attn_heads=8,
        generator=generator,
        in_dropout=0.5,
        attn_dropout=0.5,
        normalize=None,
    )
    x_inp, predictions = gat.in_out_tensors()
    model = Model(inputs=x_inp, outputs=predictions)
    model.compile(
        optimizer=optimizers.Adam(lr=0.005),
        loss=losses.categorical_crossentropy,
        metrics=["acc",metrics.Precision(),metrics.Recall()],
    )
    val_gen = generator.flow(val_subjects.index, val_targets)
    if not os.path.isdir("logs"):
        os.makedirs("logs")
    es_callback = EarlyStopping(
        monitor="val_acc", patience=20
    )  # patience is the number of epochs to wait before early stopping in case of no further improvement
    mc_callback = ModelCheckpoint(
        "logs/best_model_gat.h5", monitor="val_acc", save_best_only=True, save_weights_only=True
    )
    history = model.fit(
        train_gen,
        epochs=50,
        validation_data=val_gen,
        verbose=2,
        shuffle=False,  # this should be False, since shuffling data means shuffling the whole graph
        callbacks=[es_callback, mc_callback],
    )
    model.load_weights("logs/best_model_gat.h5")
    test_gen = generator.flow(test_subjects.index, test_targets)
    test_metrics = model.evaluate(test_gen)
    for name, val in zip(model.metrics_names, test_metrics):
        print("\t{}: {:0.4f}".format(name, val))
    if test_metrics[2]+test_metrics[3]==0:
        f1_score='inf'
    else:
        f1_score=(2*test_metrics[2]*test_metrics[3])/(test_metrics[2]+test_metrics[3])
    print("f1_score:",f1_score)
    
def my_graphsage(G,node_subjects):
    train_subjects, test_subjects = model_selection.train_test_split(node_subjects, train_size=0.1, test_size=None, stratify=node_subjects)
    target_encoding = preprocessing.LabelBinarizer()

    train_targets = target_encoding.fit_transform(train_subjects)
    test_targets = target_encoding.transform(test_subjects)
    batch_size = 50
    num_samples = [10, 5]
    generator = GraphSAGENodeGenerator(G, batch_size, num_samples)
    train_gen = generator.flow(train_subjects.index, train_targets, shuffle=True)
    graphsage_model = GraphSAGE(
    layer_sizes=[32, 32], generator=generator, bias=True, dropout=0.5,
    )
    x_inp, x_out = graphsage_model.in_out_tensors()
    prediction = layers.Dense(units=train_targets.shape[1], activation="softmax")(x_out)
    model = Model(inputs=x_inp, outputs=prediction)
    model.compile(
        optimizer=optimizers.Adam(lr=0.005),
        loss=losses.categorical_crossentropy,
        metrics=["acc",metrics.Precision(),metrics.Recall()],
    )
    test_gen = generator.flow(test_subjects.index, test_targets)
    history = model.fit(
        train_gen, epochs=20, validation_data=test_gen, verbose=2, shuffle=False
    )
    test_metrics = model.evaluate(test_gen)
    for name, val in zip(model.metrics_names, test_metrics):
        print("\t{}: {:0.4f}".format(name, val))
    if test_metrics[2]+test_metrics[3]==0:
        f1_score='inf'
    else:
        f1_score=(2*test_metrics[2]*test_metrics[3])/(test_metrics[2]+test_metrics[3])
    print("f1_score:",f1_score)
    #test_metrics = model.evaluate(test_gen)
    #print("\nTest Set Metrics:")
    #for name, val in zip(model.metrics_names, test_metrics):
        #print("\t{}: {:0.4f}".format(name, val))


In [27]:
my_gcn(G,node_subjects)

Using GCN (local pooling) filters...
Epoch 1/200
1/1 - 3s - loss: 1.9887 - acc: 0.1500 - precision_4: 0.0000e+00 - recall_4: 0.0000e+00 - val_loss: 1.9155 - val_acc: 0.1780 - val_precision_4: 0.0000e+00 - val_recall_4: 0.0000e+00 - 3s/epoch - 3s/step
Epoch 2/200
1/1 - 0s - loss: 1.9540 - acc: 0.1143 - precision_4: 0.0000e+00 - recall_4: 0.0000e+00 - val_loss: 1.9005 - val_acc: 0.2480 - val_precision_4: 0.0000e+00 - val_recall_4: 0.0000e+00 - 81ms/epoch - 81ms/step
Epoch 3/200
1/1 - 0s - loss: 1.9181 - acc: 0.2357 - precision_4: 0.0000e+00 - recall_4: 0.0000e+00 - val_loss: 1.8903 - val_acc: 0.2720 - val_precision_4: 0.0000e+00 - val_recall_4: 0.0000e+00 - 82ms/epoch - 82ms/step
Epoch 4/200
1/1 - 0s - loss: 1.8915 - acc: 0.2143 - precision_4: 0.0000e+00 - recall_4: 0.0000e+00 - val_loss: 1.8815 - val_acc: 0.3020 - val_precision_4: 0.0000e+00 - val_recall_4: 0.0000e+00 - 81ms/epoch - 81ms/step
Epoch 5/200
1/1 - 0s - loss: 1.8933 - acc: 0.2857 - precision_4: 0.0000e+00 - recall_4: 0.0000e

In [24]:
my_sgn(G,node_subjects)

Calculating 2-th power of normalized A...
1/1 [==============================] - 0s 59ms/step - loss: 2.0037 - acc: 0.1489 - precision_1: 0.0000e+00 - recall_1: 0.0000e+00
	loss: 2.0037
	acc: 0.1489
	precision_1: 0.0000
	recall_1: 0.0000
f1_score: inf


In [25]:
my_gat(G,node_subjects)

/opt/conda/lib/python3.7/site-packages/keras/initializers/initializers_v2.py:121: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  f"The initializer {self.__class__.__name__} is unseeded "


Epoch 1/50
1/1 - 8s - loss: 2.2857 - acc: 0.1500 - precision_2: 0.1053 - recall_2: 0.0143 - val_loss: 1.9488 - val_acc: 0.1260 - val_precision_2: 0.0000e+00 - val_recall_2: 0.0000e+00 - 8s/epoch - 8s/step
Epoch 2/50
1/1 - 0s - loss: 2.1747 - acc: 0.1286 - precision_2: 0.1429 - recall_2: 0.0286 - val_loss: 1.9437 - val_acc: 0.1340 - val_precision_2: 0.0000e+00 - val_recall_2: 0.0000e+00 - 186ms/epoch - 186ms/step
Epoch 3/50
1/1 - 0s - loss: 2.3543 - acc: 0.1357 - precision_2: 0.1053 - recall_2: 0.0143 - val_loss: 1.9388 - val_acc: 0.1320 - val_precision_2: 0.0000e+00 - val_recall_2: 0.0000e+00 - 127ms/epoch - 127ms/step
Epoch 4/50
1/1 - 0s - loss: 2.3094 - acc: 0.1571 - precision_2: 0.0833 - recall_2: 0.0143 - val_loss: 1.9341 - val_acc: 0.1460 - val_precision_2: 0.0000e+00 - val_recall_2: 0.0000e+00 - 156ms/epoch - 156ms/step
Epoch 5/50
1/1 - 0s - loss: 2.0947 - acc: 0.1857 - precision_2: 0.1765 - recall_2: 0.0214 - val_loss: 1.9299 - val_acc: 0.1460 - val_precision_2: 0.0000e+00 - val

In [26]:
my_graphsage(G,node_subjects)

/opt/conda/lib/python3.7/site-packages/keras/initializers/initializers_v2.py:121: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  f"The initializer {self.__class__.__name__} is unseeded "


Epoch 1/20
6/6 - 4s - loss: 1.9482 - acc: 0.1370 - precision_3: 0.0000e+00 - recall_3: 0.0000e+00 - val_loss: 1.9354 - val_acc: 0.1628 - val_precision_3: 0.0000e+00 - val_recall_3: 0.0000e+00 - 4s/epoch - 610ms/step
Epoch 2/20
6/6 - 1s - loss: 1.9367 - acc: 0.1741 - precision_3: 0.0000e+00 - recall_3: 0.0000e+00 - val_loss: 1.9143 - val_acc: 0.2129 - val_precision_3: 0.0000e+00 - val_recall_3: 0.0000e+00 - 1s/epoch - 238ms/step
Epoch 3/20
6/6 - 1s - loss: 1.9176 - acc: 0.2148 - precision_3: 0.0000e+00 - recall_3: 0.0000e+00 - val_loss: 1.8979 - val_acc: 0.2683 - val_precision_3: 0.0000e+00 - val_recall_3: 0.0000e+00 - 1s/epoch - 238ms/step
Epoch 4/20
6/6 - 1s - loss: 1.8972 - acc: 0.2111 - precision_3: 0.0000e+00 - recall_3: 0.0000e+00 - val_loss: 1.8831 - val_acc: 0.2912 - val_precision_3: 0.0000e+00 - val_recall_3: 0.0000e+00 - 1s/epoch - 238ms/step
Epoch 5/20
6/6 - 1s - loss: 1.8978 - acc: 0.2519 - precision_3: 0.0000e+00 - recall_3: 0.0000e+00 - val_loss: 1.8720 - val_acc: 0.2966 -